In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import joblib

# 1. Cargar datos
data = pd.read_csv('zonaprop_propiedades.csv')


In [2]:
# 2. Procesar tipo de vivienda
def obtener_clase(valor):
    if pd.isnull(valor):
        return None
    return valor.split('·')[0].strip()
data['vivienda'] = data['title'].apply(obtener_clase)


In [3]:
# 3. Procesar precio
ARS_TO_USD = 1 / 1200 # Tasa de conversión aproximada, ajustar según sea necesario
def convertir_precio(valor):
    if pd.isnull(valor):
        return None
    try:
        partes = valor.split()
        moneda = partes[0]
        numero_str = partes[1].replace('.', '').replace(',', '')
        numero = int(numero_str)
        if moneda == 'USD':
            return numero
        elif moneda == 'ARS':
            return int(numero * ARS_TO_USD)
        else:
            return None
    except:
        return None
data['price'] = data['rent_price'].apply(convertir_precio)


In [4]:
# 4. Procesar expensas
def convertir_expensas(valor):
    if pd.isnull(valor):
        return None
    if "No disponible" in valor:
        return None
    try:
        numero_str = valor.replace('Expensas $', '').strip().replace('.', '').replace(',', '')
        numero = int(numero_str)
        return numero
    except:
        return None
data['expenses'] = data['expenses_price'].apply(convertir_expensas)


In [5]:
# 5. Extraer variables numéricas
def extraer_numero_regex(valor):
    if pd.isnull(valor):
        return None
    try:
        match = re.search(r'(\d+(?:[.,]\d+)?)', valor)
        if match:
            numero_str = match.group(1).replace(',', '.')
            if '.' in numero_str:
                return float(numero_str)
            else:
                return int(numero_str)
        else:
            return None
    except:
        return None

columnas_mapeo = {
    'icon-stotal': 'm2_totales',
    'icon-scubierta': 'm2_cubiertos',
    'icon-ambiente': 'ambientes',
    'icon-bano': 'baños',
    'icon-cochera': 'cocheras',
    'icon-dormitorio': 'dormitorios'
}
for col_original, col_nueva in columnas_mapeo.items():
    data[col_nueva] = data[col_original].apply(extraer_numero_regex)


In [6]:
# 6. Procesar antigüedad
def convertir_antiguedad(valor):
    if pd.isnull(valor):
        return None
    if 'A estrenar' in valor:
        return 0
    try:
        match = re.search(r'\d+', valor)
        if match:
            return int(match.group(0))
        else:
            return None
    except:
        return None
data['antiguedad'] = data['icon-antiguedad'].apply(convertir_antiguedad)


In [7]:
# 7. Procesar features generales
data['general_features'] = data['general_features'].fillna('').astype(str)
def extract_plants(text):
    match = re.search(r'Cantidad plantas\s*:\s*(\d+|5 o más)', text)
    if match:
        if match.group(1) == '5 o más':
            return 5
        return int(match.group(1))
    return 1
def has_pool(text):
    return 'sí' if 'Pileta' in text else 'no'
def is_credit_compatible(text):
    return 'sí' if 'Apto profesional' in text else 'no'
data['Cantidad_plantas'] = data['general_features'].apply(extract_plants)
data['Pileta'] = data['general_features'].apply(has_pool)
data['Apto_credito'] = data['general_features'].apply(is_credit_compatible)


In [8]:
# 8. Seleccionar columnas relevantes
columns_to_keep = [
    'Cantidad_plantas', 'Pileta', 'Apto_credito', 'antiguedad', 'dormitorios',
    'cocheras', 'baños', 'ambientes', 'm2_totales', 'm2_cubiertos', 'expenses',
    'price', 'vivienda'
]
data = data[columns_to_keep]


In [9]:
# 9. Eliminar outliers extremos (3*IQR)
def get_extreme_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers
columns_numeric = ['price', 'm2_totales', 'm2_cubiertos', 'dormitorios', 'baños', 'antiguedad']
extreme_outliers = pd.DataFrame()
for column in columns_numeric:
    outliers = get_extreme_outliers(data, column)
    extreme_outliers = pd.concat([extreme_outliers, outliers])
extreme_outliers = extreme_outliers.drop_duplicates()
data = data.drop(index=extreme_outliers.index)


In [10]:
# 10. Eliminar duplicados
data = data.drop_duplicates()


In [11]:
# 11. Eliminar filas con "No disponible" en vivienda
data = data[~data["vivienda"].str.strip().str.lower().eq("no disponible")]


In [12]:
# 12. Imputar valores nulos
columnas_media = ['antiguedad', 'dormitorios', 'baños', 'ambientes', 'm2_totales', 'm2_cubiertos','expenses']
mask_casas = data['vivienda'].str.contains('Casa', case=False, na=False)
data.loc[mask_casas, 'expenses'] = data.loc[mask_casas, 'expenses'].fillna(0)
for columna in columnas_media:
    data[columna] = data.groupby('vivienda')[columna].transform(lambda x: x.fillna(round(x.mean(), 0)))
data['cocheras'] = data['cocheras'].fillna(0)
data = data.dropna(subset=['price'])


In [13]:
# 13. Estandarizar variables numéricas, menos precio porque es la variable objetivo y es logarítmica
columnas_salida = ['Cantidad_plantas', 'antiguedad', 'dormitorios', 'cocheras', 'baños',
                   'ambientes', 'm2_totales', 'm2_cubiertos', 'expenses']
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(data[columnas_salida])
df_scaled = pd.DataFrame(scaled_values, columns=columnas_salida, index=data.index)
data[columnas_salida] = df_scaled

#Logaritmo en la variable objetivo 'price'
data['price'] = np.log1p(data['price'])


In [14]:
# 14. Codificar variables categóricas
columnas_categoricas = ["Pileta","Apto_credito","vivienda"]
onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop="first")
encoded_data = onehot_encoder.fit_transform(data[columnas_categoricas])
feature_names = onehot_encoder.get_feature_names_out(columnas_categoricas)
encoded_df = pd.DataFrame(encoded_data, columns=feature_names, index=data.index)
data = data.drop(columns=columnas_categoricas)
data = pd.concat([data, encoded_df], axis=1)


In [15]:
# 15. Guardar el dataset final y los objetos de transformación
data.to_csv("dataset.csv", index=False)
joblib.dump(scaler, 'models/minmaxscaler.joblib')
joblib.dump(onehot_encoder, 'models/onehotencoder.joblib')
print("Preprocesamiento finalizado y archivos guardados.")


Preprocesamiento finalizado y archivos guardados.
